# 2. Collecte de données externes

Afin d'améliorer les performances du modèle de prédiction des prix d'occasion des vehicules (Voir le notebook 1_DataPreExploration), nous avons décidé de collecter des données externes. Cette collecte se base sur le scraping du site [Cardidiac](https://www.caradisiac.com), de la société Car&Boat Media, et qui est consacré à l'automobile (Voir [Wikipedia](https://fr.wikipedia.org/wiki/Caradisiac)).<br>
Nous avons choisi ce site pour son exhaustivité des informations que nous pouvons trouver concernant la description des vehicules ainsi que le nombre de véhicules décrits.

**Objectifs :**
* Scraper le site [Cardidiac](https://www.caradisiac.com) et récupérer les données présentes dans les fiches techniques des differentes versions de vehiculestelle que cette [Citroen C3](https://www.caradisiac.com/fiches-techniques/modele--citroen-c3-aircross/2020/1.2+puretech+110+6cv+s%5Es+feel+business+bv6/).

**Plan :** <br>
Le scraping est réalisé par plusieurs étapes. Un niveau d l'arborissance du site est parcourue à chaque étape en commencant par la page www.caradisiac.com/constructeurs--automobiles/ jusqu'aux pages des fiches techniques pour chaque versions d'un modèle (exemple www.caradisiac.com/fiches-techniques/modele--XXXXX/YEAR/YYYYYYY/ où: XXXXXX est le modèle , YEAR est l'année de commercialisation et YYYYYY est la version).
1. Collecte noms et URLs des constructeurs automobiles
2. Collecte des URLs de l'onglet 'Fiches techniques' pour chaque Modèles et Gammes
3. Collecte des URLs des Fiches techniques par année de commercialisation
4. Collecte des URLs de fiches techniques des versions présent sur les première pages
    * 4.1. Collecte des URLs des fiches technique des Versions
    * 4.2. Collecte des information présente sur les fiches technique des Versions
5. Collecte des URLs de fiches techniques des versions présent sur les pages suivantes
    * 5.1. Collecte des URLs des fiches technique des Versions
    * 5.2. Collecte des information présente sur les fiches technique des Versions
6. Conclusion
7. Prochaines étapes

In [619]:
import pandas as pd
import re

import urllib.request
from bs4 import BeautifulSoup

## 1. Collecte noms et URLs des constructeurs automobiles

In [200]:
html = urllib.request.urlopen('https://www.caradisiac.com/constructeurs--automobiles/').read()
soup = BeautifulSoup(html, 'html.parser')

div_alph_tables = soup.find_all('div', class_="ancreLettre")

list_ids = []
for item in div_alph_tables:  
    list_ids.append(item.attrs['data-letter'])

brands = []

for list_id in list_ids:
    div_name = soup.find_all('div', id=list_id)
    liste = []
    for item in div_name:  
        liste.append(item.find_all(lambda tag: tag.name == "a", {"class" : "hidden-xs"} ))
                         
    for item in liste[0]:
        brand = {}
        brand['name'] = item.attrs['title'].lower().replace(" ", "-")
        brand['href'] = item.attrs['href'].lower().replace("/modeles/","/").replace("/avis/","/").replace("/photo/","/")
        brands.append(brand)

In [186]:
len(brands)

352

Nous avons 352 constructeurs listés sur le site

## 2. Collecte des URLs des onglets 'Fiches techniques' pour chaque Modèles et Gammes

On récupère les URLs des onglets 'Fiches techniques' pour chaque Modèles et/ou Gammes présent sur l'onglet www.caradisiac.com/auto--CONSTRUCTEUR/models (CONSTRUCTEUR est le nom du constructeur automobile) en se basant sur les URLs de constructeurs collectées dans l'étapes précédante.

In [443]:
base = 'https://www.caradisiac.com'
def gamme_func(gammes, name, time, model_gamme, href):
    html = urllib.request.urlopen(base+href).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    models = soup.find_all(lambda tag: tag.name == "div", {"class" : "visuModeleGalerie"})
    for elem in models:
        if any(substring in elem.find(lambda tag: tag.name == "a").attrs['href'] for substring in ["/photo", "/avis"]):
            href_model = ""
        else:
            href_model = elem.find(lambda tag: tag.name == "a").attrs['href']
        gamme = {}
        gamme['brand'] = name
        gamme['time'] = time
        gamme['model_gamme'] = model_gamme
        gamme['model_gamme_url']= href
        
        gamme['model'] = elem.find(lambda tag: tag.name == "a").attrs['title'].lower()
        gamme['model_url'] = href_model

        gammes.append(gamme)

In [446]:
gammes = []
#b = []
#b.append({'name': 'ac', 'href': '/auto--ac/'})
#b.append({'name': 'alpina', 'href': '/auto--alpina/'})
for brand in brands:
    print(brand['href'])
    html = urllib.request.urlopen(base+brand['href']+'modeles/').read()
    soup = BeautifulSoup(html, 'html.parser')

    section_gammes_tables = soup.find_all('section', class_="borderTop")

    for item in section_gammes_tables:
        gamme_time = {}
        time = item.find(lambda tag: tag.name == "h2").getText()
        #gamme['brand'] = brand['name']
        #gamme['time'] = time
        for item in item.find_all(lambda tag: tag.name == "div", {"class" : "visuModeleGalerie"}):
            for a in item.find_all(lambda tag: tag.name == "a"):
                if any(substring in a.attrs['href'] for substring in ["/photo", "/avis"]):
                    href = ""
                else:
                    href = a.attrs['href']
                    
                if ("/gamme" in href):
                    gamme_func(gammes,brand['name'], time, a.attrs['title'].lower() ,href)
                else:
                    gamme = {}
                    gamme['brand'] = brand['name']
                    gamme['time'] = time
                    gamme['model_gamme'] = a.attrs['title'].lower()#""
                    gamme['model_gamme_url']= href#""
                    gamme['model'] = a.attrs['title'].lower()
                    gamme['model_url'] = href
                    gammes.append(gamme)

/auto--4-stroke/
/auto--abarth/
/auto--ac/
/auto--acrea/
/auto--acura/
/auto--ad-tramontana/
/auto--adler/
/auto--aiways/
/auto--aixam/
/auto--al-melling/
/auto--aleko/
/auto--alfa-romeo/
/auto--alke/
/auto--alpina/
/auto--alpine-renault/
/auto--alvis/
/auto--amc/
/auto--amphicar/
/marque-moto--anaig/
/marque-moto--apal/
/auto--aptera-motors/
/auto--ariel/
/auto--armstrong-siddeley/
/auto--aro/
/auto--artega/
/auto--ascari/
/auto--aspark/
/auto--aston-martin/
/marque-moto--auburn/
/auto--audi/
/auto--austin/
/auto--austin-healey/
/auto--auto-union/
/auto--autobianchi/
/auto--auverland/
/auto--bee-automobiles/
/auto--bellier/
/auto--bentley/
/auto--berliet/
/auto--bertone/
/auto--bitter/
/auto--bizzarrini/
/auto--bmw/
/auto--bollore/
/auto--bongarini/
/auto--booxt/
/auto--borgward/
/auto--bowler/
/auto--brabus/
/auto--bremach/
/auto--brilliance/
/auto--bristol/
/auto--brooke-cars/
/auto--bugatti/
/auto--buick/
/auto--burby-s/
/auto--byd/
/auto--cadillac/
/auto--caparo/
/auto--carbodies/

In [519]:
#Sauvegarde du résultats
models = pd.DataFrame.from_dict(gammes)
models.to_csv("../scrapping_models.csv")

## 3. Collecte des URLs des Fiches techniques par année de commercialisation

On récupère les URLs des fiches techniques pour chaque version de vehicule, par année de commercialisation, présentes sur www.caradisiac.com/fiches-techniques/modele--MODEL/ (MODEL est le modèle du vehicule).

In [607]:
fiche_tech = []
for item in gammes:
    model_url = item['model_url']
    if model_url=="":
        fich_tech_href = ""
    else:
        if "/fiche" in model_url:
            fich_tech_href = model_url
        else:
            html = urllib.request.urlopen(base+model_url).read()
            soup = BeautifulSoup(html, 'html.parser')
            a_fiche_tech = soup.find('a', class_="twoLine", text="Fiches techniques")
            fich_tech_href = ''
            if a_fiche_tech is not None:
                fich_tech_href = a_fiche_tech.get('href')
        item['fich_tech_href'] = fich_tech_href
        print(item['brand'])
    fiche_tech.append(item)

In [606]:
#models_fiche_tech_urls = pd.DataFrame.from_dict(fiche_tech)
models_fiche_tech_urls.to_csv("../scrapping_models_fiche_tech_urls_all.csv")

In [628]:
models_fiche_tech_urls_exist = models_fiche_tech_urls.loc[(models_fiche_tech_urls['fich_tech_href'].str.contains("/fiches-"))].copy()

In [767]:
models_fiche_tech_urls_exist.drop_duplicates().shape

(2310, 7)

In [605]:
#save just rows where fiches exists
models_fiche_tech_urls_exist.to_csv("../scrapping_models_fiche_tech_urls_exist.csv")

## 4. Collecte des URLs de fiches techniques des versions présent sur les première pages

Collecte de la listes des URLs renvoyant vers les fiches techniques de chaque version. Certains modèles ayant plus de 200 versions ont plusieurs pages listant les URLs vers les fiches techniques des versions. Dans, cette partie, on récupère uniquement les URLs présentes sur les premières pages.

### 4.1. Collecte des URLs des fiches technique des Versions

In [654]:
fiche_tech_urls[0]

{'brand': 'abarth',
 'time': "Les modèles d'aujourd'hui",
 'model_gamme': 'abarth 124',
 'model_gamme_url': '/modele--abarth-124-2e-generation-spider/',
 'model': 'abarth 124',
 'model_url': '/modele--abarth-124-2e-generation-spider/',
 'fich_tech_href': '/fiches-techniques/modele--abarth-124-2e-generation-spider/'}

In [1470]:
fiche_tech_urls = models_fiche_tech_urls_exist.to_dict(orient='records')

fich_year_urls = []
for item in fiche_tech_urls:
    fich_tech_url = item['fich_tech_href']
    html = urllib.request.urlopen(base+fich_tech_url).read()
    soup = BeautifulSoup(html, 'html.parser')

    fiche_tech_list = soup.find_all(lambda tag: tag.name == "li", {"class" : "font16"})
    for elem in fiche_tech_list:
        temp = item.copy()
        temp['year'] = re.findall(r'\d+$', elem.find(lambda tag: tag.name == "a").getText())[0]
        temp['year_url'] = elem.find(lambda tag: tag.name == "a").attrs['href']
        fich_year_urls.append(temp)
        #print((temp['year'],temp['year_url']))

In [669]:
len(fich_year_urls)

16902

In [670]:
models_fiche_tech_year_urls = pd.DataFrame.from_dict(fich_year_urls)
models_fiche_tech_year_urls.to_csv("../scrapping_models_fiche_tech_year_urls.csv")

In [1469]:
version_urls = []
for item in fich_year_urls:
    fich_year_url = item['year_url']
    html = urllib.request.urlopen(base+fich_year_url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    fiche_version_list = soup.find_all(lambda tag: tag.name == "a", {"onclick":"xt_click(this, 'C',xtn2,id_context+'::'+id_page+'::ListeVersion_Version_XTC', 'N');"})
    for elem in fiche_version_list :
        temp = item.copy()
        temp['version_name'] = elem.getText()
        temp['version_url'] = elem.attrs['href']
        version_urls.append(temp)
    #print((item['year'],item['year_url']))

In [738]:
model_version_urls = pd.DataFrame.from_dict(version_urls)
model_version_urls.to_csv("../scrapping_models_versions_url.csv")

In [773]:
#model_version_urls = model_version_urls_bis

In [887]:
model_version_urls_unique_without_year_next = model_version_urls.drop_duplicates(subset=['model_gamme','model',
                                                'model_gamme_url','model_url',
                                                'fich_tech_href','version_name',
                                               'version_url'], ignore_index=True)

In [1302]:
model_version_urls_unique_without_year_next.to_csv("../Final_scrapping_model_version_urls_unique_without_year_next.csv")

### 4.2. Collecte des information présente sur les fiches technique des Versions

In [889]:
model_version_urls_unique_without_year_next.shape, model_version_urls.shape

((123603, 11), (351400, 11))

#### Note :
A cause des connexions interrempues d'un moment à l'autre, nous avons executé le code suivant plusieurs fois en concatinant à chaque fois les résultats stockés dans des fichiers CSV.

In [ ]:
version_urls_unique_without_year_next_bis = []
#version_urls_unique_without_year_next_bis.append(version_urls_unique_without_year_next[44242])
#version_urls_unique_without_year_next_bis.append(version_urls_unique_without_year_next[44243])

for i in range(0,len(version_urls_unique_without_year_next)):
    if i >= 97905:
        version_urls_unique_without_year_next_bis.append(version_urls_unique_without_year_next[i])

In [899]:
#version_urls_unique_without_year_next_bis# = model_version_urls_unique_without_year_next.to_dict(orient='records')
info_urls = []
for item in version_urls_unique_without_year_next:
    vers_url = item['version_url']
    html = urllib.request.urlopen(base+vers_url).read()
    soup = BeautifulSoup(html, 'html.parser')
    #print(html)
    # Extraction du prix
    prix_span = soup.find(lambda tag: tag.name == "span", {"class":"blocPrixVal"})
    #print(prix_span)
    prix =""
    #if prix_span is not None:
    prix = prix_span.getText()
    #Taux d'émission de CO2 et bonus/malus (Norme NEDC)
    #taux_co2 = soup.find(lambda tag: tag.name == "p", {"class":"valueEmis malus"}).getText()
    
    # Extraction de caractéristiques
    fiche_tech_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_1"})
    first = fiche_tech_div.find_all(lambda tag: tag.name == "td", {"class":"intitule first"})
    last = fiche_tech_div.find_all(lambda tag: tag.name == "td", {"class":"last result"})
    cara = []
    for i in range(0, len(first)) :
        temp = {}
        #print((first[i].getText(),last[i].getText()))
        temp[first[i].getText()] = last[i].getText()
        cara.append(temp)
     
    # Extraction d'équipement
    equip_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_2"})
    equip_tables = equip_div.find_all(lambda tag: tag.name == "table", {"class":"listingTab"})
    equips_all = []
    for table in equip_tables:
        h3 = table.find(lambda tag: tag.name == "h3")
        equips_td = table.find_all(lambda tag: tag.name == "td", {"class":"intitule first"})
        title = h3.getText()
        equips = []
        for elem in equips_td:
            equips.append(elem.getText())
        temp = {}
        temp[title] = equips
        equips_all.append(temp) 
        
    # Extraction d'option
    option_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_3"})
    first = option_div.find_all(lambda tag: tag.name == "td", {"class":"intitule first"})
    last = option_div.find_all(lambda tag: tag.name == "td", {"class":"last result"})
    options = []
    for i in range(0, len(first)) :
        option = {}
        #print((first[i].getText(),last[i].getText()))
        option[first[i].getText()] = last[i].getText()
        options.append(option)
    
    # Extraction de couleur
    #couleur_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_4"})
    #couleurs_td = couleur_div.find_all(lambda tag: tag.name == "td", {"class":"intitule"})
    #couleurs = []
    #for elem in couleurs_td:
    #    couleurs.append(elem.getText())
    temp = item.copy()
    temp['prix']= prix
    temp['caracteristiques'] = cara
    temp['equips_all'] = equips_all
    temp['options'] = options 
    info_urls.append(temp)
    
    print((temp['Unnamed: 0'], temp['model']))
    #print(prix)
    #print(cara)
    #print(equips_all)
    #print(options)
    #print(couleurs)
#info_urls 

('abarth 124', 'II 1.4 TURBO 170', '/fiches-techniques/modele--abarth-124-2e-generation-spider/2016/ii+1.4+turbo+170/')
('abarth 124', 'II 1.4 TURBO 170 GT', '/fiches-techniques/modele--abarth-124-2e-generation-spider/2018/ii+1.4+turbo+170+gt/')


In [ ]:
model_info_urls = pd.DataFrame.from_dict(info_urls)
model_info_urls.to_csv("Caradisiac/scrapping_caradisiac_14.csv")
model_info_urls.shape

## 5. Collecte des URLs de fiches techniques des versions présent sur les pages suivantes
Collecte des URLs des fiches techniques des versions qui se trouvent sur les pages suivantes dans le cas où ces dernières existent

In [925]:
model_unique_year_urls = model_version_urls.drop_duplicates(subset=["year_url"])

### 5.1. Collecte des URLs des fiches technique des Versions

In [1190]:
def get_next(tab,elem, page):
    html = urllib.request.urlopen(page).read()
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.find(lambda tag: tag.name == "a", {"class" : "floatR pageSuiv JSFormFTPage"})
    #print(page)
    next_page = []
    if a is not None:
        page = page.split("?")[0]
        next_page = a.attrs["href"]
        if(isinstance(next_page, str)):
            next_url = page+next_page
            #print(next_page)
            temp = elem.copy()
            result = get_next(tab, elem, next_url)
            if (result!=""):
                temp["next_page"] = result
                next_pages_urls_again.append(temp)
            return next_page
        else:
            return ""
    else:
        return ""

In [1468]:
next_pages_urls_again = []
for elem in unique_year_urls:
    
    temp = elem.copy()
    result = get_next(next_pages_urls_again,elem, base+elem["year_url"])
    #next_pages_urls_again.append(elem)
    if (result!=""):
        temp["next_page"] = result
        next_pages_urls_again.append(temp)
    print(elem['year_url'])
    #next_pages_urls_again.append(temp)
    #print(elem["year_url"])

In [1245]:
model_next_pages_urls_exist = pd.DataFrame.from_dict(next_pages_urls_again)
model_next_pages_urls_exist.shape

(419, 12)

In [1246]:
model_next_pages_urls_exist.to_csv("../model_next_pages_urls_exist.csv")

In [1281]:
model_next_pages_urls_exist["brand"].unique().size

13

In [1279]:
len(next_pages_urls_again)

419

In [1278]:
model_next_pages_urls_exist.shape

(419, 12)

In [1467]:
version_urls_nextpages = []
for item in next_pages_urls_again:
    fich_year_url = item['year_url']+item['next_page']
    html = urllib.request.urlopen(base+fich_year_url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    fiche_version_list = soup.find_all(lambda tag: tag.name == "a", {"onclick":"xt_click(this, 'C',xtn2,id_context+'::'+id_page+'::ListeVersion_Version_XTC', 'N');"})
    for elem in fiche_version_list :
        temp = item.copy()
        temp['version_name'] = elem.getText()
        temp['version_url'] = elem.attrs['href']
        version_urls_nextpages.append(temp)
    #print((item['year'],item['year_url'], item['next_page']))

In [1293]:
model_version_urls_nextpages = pd.DataFrame.from_dict(version_urls_nextpages)

In [1311]:
model_version_urls_nextpages.shape

(59857, 12)

In [1305]:
model_version_urls_unique_without_year_next_nextpages = model_version_urls_nextpages.drop_duplicates(subset=['model_gamme','model',
                                                'model_gamme_url','model_url',
                                                'fich_tech_href','version_name',
                                               'version_url'], ignore_index=True)


In [1313]:
model_version_urls_unique_without_year_next_nextpages.to_csv("../Final_scrapping_model_version_urls_unique_without_year_nextpages.csv")

**Find the unique rows that are not in the first page:**

In [1427]:
## Find the unique rows that are not in the first page 
diffrence_between_first_vs_nextPages = pd.concat([model_version_urls_unique_without_year_next, model_version_urls_unique_without_year_next_nextpages])
diffrence_between_first_vs_nextPages.drop_duplicates(subset=['model_gamme','model',
                                                'model_gamme_url','model_url',
                                                'fich_tech_href','version_name',
                                               'version_url'], ignore_index=True)
diffrence_between_first_vs_nextPages = test.dropna(subset = ["next_page"])

In [1430]:
diffrence_between_first_vs_nextPages.shape

(15450, 12)

In [1503]:
diffrence_between_first_vs_nextPages.to_csv("../Final_scrapping_model_version_urls_unique_without_year_nextpages_not_in_first_pages.csv")


In [1502]:
diffrence_between_first_vs_nextPages["brand"].unique()

array(['audi', 'bmw', 'citroen', 'fiat', 'ford', 'iveco', 'mercedes',
       'mini', 'opel', 'peugeot', 'renault', 'volkswagen'], dtype=object)

In [1572]:
diffrence_between_first_vs_nextPages[diffrence_between_first_vs_nextPages["brand"]=="iveco"] \
["model"].value_counts()


iveco daily 5    6050
iveco daily 4     450
Name: model, dtype: int64

In [1575]:
model_version_urls_unique_without_year_next[model_version_urls_unique_without_year_next["model"]=="iveco daily 3"]["year"].unique()


array(['1999', '2000', '2002', '2003', '2009'], dtype=object)

In [1559]:
diffrence_between_first_vs_nextPages["model_gamme"].value_counts()["iveco daily"]

6500

**Example 1:**

"V (2) 2.3 33S11 V12 H2 3250L" is in page 1 of year 2019. It appears in the first pages dataset. So, it shoud not appear on the dataset of  next pages.

In [1460]:
model_version_urls_unique_without_year_next.loc[(model_version_urls_unique_without_year_next['version_name']=="V (2) 2.3 33S11 V12 H2 3250L")]


,brand,time,model_gamme,model_gamme_url,model,model_url,fich_tech_href,year,year_url,version_name,version_url
45818,iveco,Les modèles d'aujourd'hui,iveco daily,/gamme--iveco-daily/,iveco daily 5,/modele--iveco-daily-5/,/fiches-techniques/modele--iveco-daily-5/,2002,/fiches-techniques/modele--iveco-daily-5/,V (2) 2.3 33S11 V12 H2 3250L,/fiches-techniques/modele--iveco-daily-5/2019/v+(2)+2.3+33s11+v12+h2+3250l/


In [1459]:
diffrence_between_first_vs_nextPages[diffrence_between_first_vs_nextPages['version_name']=="V (2) 2.3 33S11 V12 H2 3250L"]


,brand,time,model_gamme,model_gamme_url,model,model_url,fich_tech_href,year,year_url,version_name,version_url,next_page


**Exemple 2:**

"V (2) 2.3 35C16 D/P 3450 LEAF" is in page 2 of year 2019. It doesn't appear in the first pages dataset

In [1464]:
diffrence_between_first_vs_nextPages[diffrence_between_first_vs_nextPages['version_name']=="V (2) 2.3 35C16 D/P 3450 LEAF"]


,brand,time,model_gamme,model_gamme_url,model,model_url,fich_tech_href,year,year_url,version_name,version_url,next_page
128769,iveco,Les modèles d'aujourd'hui,iveco daily,/gamme--iveco-daily/,iveco daily 5,/modele--iveco-daily-5/,/fiches-techniques/modele--iveco-daily-5/,2002,/fiches-techniques/modele--iveco-daily-5/,V (2) 2.3 35C16 D/P 3450 LEAF,/fiches-techniques/modele--iveco-daily-5/2019/v+(2)+2.3+35c16+d*p+3450+leaf/,?page=2


In [1465]:
model_version_urls_unique_without_year_next.loc[(model_version_urls_unique_without_year_next['version_name']=="V (2) 2.3 35C16 D/P 3450 LEAF")]


,brand,time,model_gamme,model_gamme_url,model,model_url,fich_tech_href,year,year_url,version_name,version_url


### 5.2. Collecte des information présente sur les fiches technique des Versions

In [ ]:
version_urls_unique_without_year_next_bis = []
#version_urls_unique_without_year_next_bis.append(version_urls_unique_without_year_next[44242])
#version_urls_unique_without_year_next_bis.append(version_urls_unique_without_year_next[44243])

for i in range(0,len(version_urls_unique_without_year_next)):
    if i >= 97905:
        version_urls_unique_without_year_next_bis.append(version_urls_unique_without_year_next[i])

In [ ]:
#version_urls_unique_without_year_next_bis# = model_version_urls_unique_without_year_next.to_dict(orient='records')
info_urls = []
for item in version_urls_unique_without_year_next:
    vers_url = item['version_url']
    html = urllib.request.urlopen(base+vers_url).read()
    soup = BeautifulSoup(html, 'html.parser')
    #print(html)
    # Extraction du prix
    prix_span = soup.find(lambda tag: tag.name == "span", {"class":"blocPrixVal"})
    #print(prix_span)
    prix =""
    #if prix_span is not None:
    prix = prix_span.getText()
    #Taux d'émission de CO2 et bonus/malus (Norme NEDC)
    #taux_co2 = soup.find(lambda tag: tag.name == "p", {"class":"valueEmis malus"}).getText()
    
    # Extraction de caractéristiques
    fiche_tech_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_1"})
    first = fiche_tech_div.find_all(lambda tag: tag.name == "td", {"class":"intitule first"})
    last = fiche_tech_div.find_all(lambda tag: tag.name == "td", {"class":"last result"})
    cara = []
    for i in range(0, len(first)) :
        temp = {}
        #print((first[i].getText(),last[i].getText()))
        temp[first[i].getText()] = last[i].getText()
        cara.append(temp)
     
    # Extraction d'équipement
    equip_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_2"})
    equip_tables = equip_div.find_all(lambda tag: tag.name == "table", {"class":"listingTab"})
    equips_all = []
    for table in equip_tables:
        h3 = table.find(lambda tag: tag.name == "h3")
        equips_td = table.find_all(lambda tag: tag.name == "td", {"class":"intitule first"})
        title = h3.getText()
        equips = []
        for elem in equips_td:
            equips.append(elem.getText())
        temp = {}
        temp[title] = equips
        equips_all.append(temp) 
        
    # Extraction d'option
    option_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_3"})
    first = option_div.find_all(lambda tag: tag.name == "td", {"class":"intitule first"})
    last = option_div.find_all(lambda tag: tag.name == "td", {"class":"last result"})
    options = []
    for i in range(0, len(first)) :
        option = {}
        #print((first[i].getText(),last[i].getText()))
        option[first[i].getText()] = last[i].getText()
        options.append(option)
    
    # Extraction de couleur
    #couleur_div = soup.find(lambda tag: tag.name == "div", {"id":"tabContent_4"})
    #couleurs_td = couleur_div.find_all(lambda tag: tag.name == "td", {"class":"intitule"})
    #couleurs = []
    #for elem in couleurs_td:
    #    couleurs.append(elem.getText())
    temp = item.copy()
    temp['prix']= prix
    temp['caracteristiques'] = cara
    temp['equips_all'] = equips_all
    temp['options'] = options 
    info_urls.append(temp)
    
    print((temp['Unnamed: 0'], temp['model']))
    #print(prix)
    #print(cara)
    #print(equips_all)
    #print(options)
    #print(couleurs)
#info_urls 

In [ ]:
model_info_urls = pd.DataFrame.from_dict(info_urls)
model_info_urls.to_csv("Caradisiac/scrapping_caradisiac_14.csv")
model_info_urls.shape

## 6. Conclusion
Dans ce notebook, nous avons récupéré les informations présentes sur les pages 'Fiches Techniques' des versions des vehicules listées dans le site caradisiac. <br>

A cause des problèmes de connexion, des datasets ont été récupérés après chaque interuption de connexion.

## 7. Prochaines étapes
* Croisement des données externes avec les données Data_cars et réalisation du data engineering.
* Entrainement du modèle RandomFroest sur les nouvelles données